**Last modified on**: 13/11/2024

**Author**: Onur Serçinoğlu

**Credits**:

This jupyter notebook is a minimally-modified version of the one prepared by Prof. Ian Simpson for the Bioinformatics I course taught at University of Edinburgh (**https://github.com/tisimpson/bioinformatics1/blob/main/labs/notebooks/bio1_week4_lab2.ipynb**)

### Note

muscle MSA program is required.

You can do this either by installing from conda:

`conda install -c bioconda muscle`

and the calling it from the terminal directly with `muscle`

or download an executable binary from the tool's repository at

https://github.com/rcedgar/muscle/releases

The latter option is used below in this notebook.

# BLAST, MSA, and Phylogenetic Trees using Biopython

In [ ]:
from Bio import Entrez
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from Bio.Align import AlignInfo
from matplotlib import pyplot as plt

## Perform a BLAST Query

In [ ]:
Entrez.email = 'osercinoglu@gtu.edu.tr'

# beta-globin, human
my_protein = 'NP_000509.1' 

handle = Entrez.efetch(db="protein", id=my_protein, rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()

# show the sequence record
print(record)

In [ ]:
result_handle = NCBIWWW.qblast('blastp', 'swissprot', record.seq)
# This may take some time to run

# parse the results
result_handle.seek(0)
blast_record = NCBIXML.read(result_handle)

In [ ]:
#print out the key information for the hits

print('Gene name\te-value\tscore')
for a in blast_record.alignments:
    print(a.title.split('|')[2].split('Full=')[1].split(';')[0]+'\t'+str(a.hsps[0].expect)+'\t'+str( a.hsps[0].score))

In [ ]:
# show the species and alignment scores
a=blast_record.alignments[0]
sp_ids = []
for a in blast_record.alignments:
    sp_ids.append(a.title.split('|')[1])
# print(",".join(sp_ids))
handle = Entrez.efetch(db="protein", id=",".join(sp_ids), retmode="xml")#, rettype='gb')
data = Entrez.read(handle)
species = []
print('Alignment score\tSpecies')
for i,d in enumerate(data):
    species.append(d['GBSeq_source'])
    print(str(blast_record.alignments[i].hsps[0].score)+'\t'+d['GBSeq_source'])

## Select sequences for MSA

In [ ]:
E_VALUE_THRESH = 1e-6

for i,alignment in enumerate(blast_record.alignments):
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence: ', alignment.title)
            print('species: '+species[i])
            print('length: ', alignment.length)
            print('e value: ', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')
            print(hsp.match[0:75] + '...')

In [ ]:
# now work with all results with e-value below this value:
E_VALUE_THRESH = 1e-6

# the following will write all results into a FASTA file for the MSA 

def get_seqrecs(alignments, threshold):
    # a little helper function to get the sequence records
    for i,aln in enumerate(alignments):
        for hsp in aln.hsps:
            if hsp.expect < threshold:
                id = species[i]
                # id = aln.title.split('|')[1].split(' ')[0].split('_')[0]+'_'+species[i].replace(' ','_')
                print(id)
                yield SeqRecord(Seq(hsp.sbjct), id=id,description=str(aln.title.split('|')[1]))
                break
 
best_seqs = get_seqrecs(blast_record.alignments, E_VALUE_THRESH)
# write out to a fasta file
SeqIO.write(best_seqs, 'blast_selected_globins.fa', 'fasta')

In [ ]:
import os

# run Muscle MSA
cmdLine = '/home/onur/software/muscle-linux-x86.v5.3 -align blast_selected_globins.fa -output blast_selected_globins_alignment.aln'
os.popen(cmdLine)

In [ ]:
#read in and then print out alignment
alignment = AlignIO.read('blast_selected_globins_alignment.aln','fasta')

# a small correct in alignment object.
for s in alignment:
    s.id = s.description
    s.name = s.description
    
print(alignment)

In [ ]:
summary_align = AlignInfo.SummaryInfo(alignment)

# compute a consensus sequence by taking the most frequent letter
# positions below a thresold similarity are shown as 'X'

# the threshold can be adjusted by adding e.g. threshold=0.5

print('Consensus sequence without gaps:')
print(summary_align.dumb_consensus())
print('Consensus sequence with gaps:')
print(summary_align.gap_consensus())

## Construct a simple phylogenetic tree

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
#from TreeConstruction import DistanceCalculator
calculator = DistanceCalculator('pam250')
dm = calculator.get_distance(alignment)
print(dm)

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
#from TreeConstruction import DistanceTreeConstructor
# here supply the keyword upgma or nj
# compare the trees you get from both methods
constructor = DistanceTreeConstructor(calculator, 'upgma')
tree = constructor.build_tree(alignment)
print(tree)

In [ ]:
from Bio import Phylo
# now draw the tree, try out these three methods:
Phylo.draw_ascii(tree)

In [ ]:
# or a nicer looking one
plt.figure(figsize=(12,12))
ax=plt.subplot(111)
Phylo.draw(tree,axes=ax)